# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(Path("..", "WeatherPy", "output_data", "cities.csv"))

# Display sample data
city_data_df

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santana,-0.0583,-51.1817,26.10,94,75,1.03,BR,1714882848
1,1,chonchi,-42.6128,-73.8092,2.84,93,100,1.90,CL,1714883073
2,2,invercargill,-46.4000,168.3500,11.68,80,73,4.92,NZ,1714883073
3,3,adamstown,-25.0660,-130.1015,21.00,64,93,8.60,PN,1714883073
4,4,saipan,15.1355,145.7010,32.39,58,20,6.17,MP,1714883073
...,...,...,...,...,...,...,...,...,...,...
583,583,petrolina,-9.3986,-40.5008,25.03,78,0,3.60,BR,1714883156
584,584,taroa,8.7058,171.2275,27.95,77,89,7.54,MH,1714883156
585,585,namsos,64.4662,11.4957,5.03,80,32,3.09,NO,1714883156
586,586,muynoq,43.7683,59.0214,16.20,58,55,2.49,UZ,1714883156


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    alpha = 0.5,
    color = 'City',
    size = 'Humidity'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[
    (city_data_df['Temperature'] <= 25) &
    (city_data_df['Humidity'] <= 70) &
    (city_data_df['Cloudiness'] <= 50)
]

# Drop any rows with null values
ideal_df.dropna(inplace=True)

# Display sample data
ideal_df

/tmp/ipykernel_396955/1733935799.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df.dropna(inplace=True)


,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,blackmans bay,-43.0167,147.3167,14.07,63,4,0.89,AU,1714883074
13,13,gardez,33.5974,69.2259,12.57,68,20,2.65,AF,1714883074
30,30,pierre,44.3683,-100.3510,11.03,62,0,3.60,US,1714883077
49,49,ivdel',60.6911,60.4206,-0.54,51,26,3.13,RU,1714883080
69,69,yafran,32.0633,12.5286,17.38,51,0,0.31,LY,1714883083
...,...,...,...,...,...,...,...,...,...,...
566,566,cova figueira,14.8905,-24.2934,18.71,69,0,4.78,CV,1714883153
569,569,yerofey pavlovich,53.9667,121.9667,15.91,25,11,0.95,RU,1714883154
575,575,turkmenbasy,40.0222,52.9552,16.43,63,0,6.17,TM,1714883154
581,581,sretensk,52.2453,117.7093,17.78,24,19,0.42,RU,1714883155


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.Series(dtype='str')

# Display sample data
hotel_df

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,blackmans bay,-43.0167,147.3167,14.07,63,4,0.89,AU,1714883074,NaN
13,13,gardez,33.5974,69.2259,12.57,68,20,2.65,AF,1714883074,NaN
30,30,pierre,44.3683,-100.3510,11.03,62,0,3.60,US,1714883077,NaN
49,49,ivdel',60.6911,60.4206,-0.54,51,26,3.13,RU,1714883080,NaN
69,69,yafran,32.0633,12.5286,17.38,51,0,0.31,LY,1714883083,NaN
...,...,...,...,...,...,...,...,...,...,...,...
566,566,cova figueira,14.8905,-24.2934,18.71,69,0,4.78,CV,1714883153,NaN
569,569,yerofey pavlovich,53.9667,121.9667,15.91,25,11,0.95,RU,1714883154,NaN
575,575,turkmenbasy,40.0222,52.9552,16.43,63,0,6.17,TM,1714883154,NaN
581,581,sretensk,52.2453,117.7093,17.78,24,19,0.42,RU,1714883155,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'limit': 1,
    'radius': radius,
    'categories': 'accommodation.hotel',
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    params['lat'] = row['Latitude']
    params['lon'] = row['Longitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{params['lon']},{params['lat']},{params['radius']}"
    params["bias"] = f"proximity:{params['lon']},{params['lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Villa Howden', 'country': 'Australia', 'country_code': 'au', 'state': 'Tasmania', 'city': 'Hobart', 'village': 'Howden', 'municipality': 'Kingborough', 'postcode': '7054', 'street': 'Howden Road', 'lon': 147.27947946462487, 'lat': -43.0097946, 'state_code': 'TAS', 'formatted': 'Villa Howden, Howden Road, Hobart TAS 7054, Australia', 'address_line1': 'Villa Howden', 'address_line2': 'Howden Road, Hobart TAS 7054, Australia', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details', 'details.contact'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'fax': '+61 3 6267 1533', 'name': 'Villa Howden', 'email': 'stay@villahowden.com.au', 'phone': '+61 3 6267 1161', 'osm_id': 618783944, 'tou

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,blackmans bay,-43.0167,147.3167,14.07,63,4,0.89,AU,1714883074,Villa Howden
13,13,gardez,33.5974,69.2259,12.57,68,20,2.65,AF,1714883074,No hotel found
30,30,pierre,44.3683,-100.3510,11.03,62,0,3.60,US,1714883077,Quality Inn
49,49,ivdel',60.6911,60.4206,-0.54,51,26,3.13,RU,1714883080,гостиница Ивдель
69,69,yafran,32.0633,12.5286,17.38,51,0,0.31,LY,1714883083,فندق يفرن السياحي
...,...,...,...,...,...,...,...,...,...,...,...
566,566,cova figueira,14.8905,-24.2934,18.71,69,0,4.78,CV,1714883153,No hotel found
569,569,yerofey pavlovich,53.9667,121.9667,15.91,25,11,0.95,RU,1714883154,No hotel found
575,575,turkmenbasy,40.0222,52.9552,16.43,63,0,6.17,TM,1714883154,Hazar Myhmanhanasy
581,581,sretensk,52.2453,117.7093,17.78,24,19,0.42,RU,1714883155,No hotel found


In [53]:
hotel_df

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,blackmans bay,-43.0167,147.3167,14.07,63,4,0.89,AU,1714883074,Villa Howden
13,13,gardez,33.5974,69.2259,12.57,68,20,2.65,AF,1714883074,No hotel found
30,30,pierre,44.3683,-100.3510,11.03,62,0,3.60,US,1714883077,Quality Inn
49,49,ivdel',60.6911,60.4206,-0.54,51,26,3.13,RU,1714883080,гостиница Ивдель
69,69,yafran,32.0633,12.5286,17.38,51,0,0.31,LY,1714883083,فندق يفرن السياحي
...,...,...,...,...,...,...,...,...,...,...,...
566,566,cova figueira,14.8905,-24.2934,18.71,69,0,4.78,CV,1714883153,No hotel found
569,569,yerofey pavlovich,53.9667,121.9667,15.91,25,11,0.95,RU,1714883154,No hotel found
575,575,turkmenbasy,40.0222,52.9552,16.43,63,0,6.17,TM,1714883154,Hazar Myhmanhanasy
581,581,sretensk,52.2453,117.7093,17.78,24,19,0.42,RU,1714883155,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    alpha = 0.5,
    color = 'City',
    size = 'Humidity',
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)